In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_cross_validation_metric
from tqdm import tqdm
import os
from pathlib import Path
import sys
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio
from prophet.serialize import model_to_json, model_from_json
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose, STL

from statsmodels.tsa.stattools import adfuller, acf, pacf, q_stat
from scipy.stats import boxcox

import shutil

# Set default Plotly theme to have a white background
pio.templates.default = "plotly_white"

# Set default Matplotlib style to have a white background
plt.style.use("default")


In [ ]:
# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    # mape = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1))) * 100
    return rmse


# Extract events from the calendar data
def extract_holidays(calendar):
    # Extract relevant columns and filter rows with non-null events
    events = calendar[
        (calendar["event_name_1"].notna()) | (calendar["event_name_2"].notna())
    ][["date", "event_name_1", "event_name_2"]]

    # Reshape to long format
    events_long = pd.melt(
        events,
        id_vars=["date"],
        value_vars=["event_name_1", "event_name_2"],
        var_name="event_type",
        value_name="event",
    )
    events_long = events_long.dropna(subset=["event"])  # Drop rows without events

    # Rename columns to match Prophet's expectations
    holidays = (
        events_long.rename(columns={"date": "ds", "event": "holiday"})
        .drop(columns=["event_type"])
        .drop_duplicates()
    )

    return holidays


# Define a function to transform and clean data
def transform_and_clean_data(sales_data, start_date="2011-01-29"):
    sales_long = sales_data.melt(id_vars=["id"], var_name="day_id", value_name="sales")
    sales_long["day_num"] = sales_long["day_id"].str.extract(r"(\d+)").astype(int)
    sales_long["date"] = pd.to_datetime(start_date) + pd.to_timedelta(
        sales_long["day_num"] - 1, unit="d"
    )
    maxDate = sales_long["date"].dt.strftime("%Y-%m-%d").unique().max()
    return sales_long[["id", "date", "sales"]], maxDate


In [ ]:
# Load data
calendar = pd.read_csv("data/calendar_afcs2024.csv")
prices = pd.read_csv("data/sell_prices_afcs2024.csv")
sales_train = pd.read_csv("data/sales_train_validation_afcs2024.csv")
sales_validation = pd.read_csv("data/sales_test_validation_afcs2024.csv")
sales_test = pd.read_csv("data/sales_test_evaluation_afcs2024.csv")


# Transform training data
cleaned_train_sales_data, last_date = transform_and_clean_data(
    sales_train, start_date="2011-01-29"
)
# display(cleaned_train_sales_data.head())
# print(last_date)

cleaned_validation_sales_data, last_date = transform_and_clean_data(
    sales_validation, start_date="2011-01-29"
)
# display(cleaned_validation_sales_data.head())
# print(last_date)

# # Prepare the test data
cleaned_test_sales_data, last_day = transform_and_clean_data(
    sales_test, start_date="2011-01-29"
)
# display(cleaned_test_sales_data.head())
# print(last_day)

# Merge sales_validation and sales_train for training
merged_train_data = pd.concat(
    [cleaned_train_sales_data, cleaned_validation_sales_data],
)
display(merged_train_data.head())


### SET THE NUMBER OF MODELS

Using K or "all"


In [ ]:
# Extract unique product IDs
product_ids = merged_train_data["id"].unique()

# Calculate total sales for each product
total_sales = (
    merged_train_data.groupby("id")["sales"]
    .sum()
    .reset_index()
    .rename(columns={"sales": "total_sales"})
).sort_values(by="total_sales", ascending=False)
display(total_sales.head())


# Get top K products based on total sales
top_k = "all"


if isinstance(top_k, int):
    products = total_sales.nlargest(top_k, "total_sales")["id"].tolist()
elif top_k == "all":
    products = product_ids


In [ ]:
def forecast_with_exponential_smoothing(
    train_data, test_data, product_dir, smoothing_level=None
):
    """
    Forecast sales using Simple Exponential Smoothing and save the forecast plot.
    """

    # Create a copy of the test data to store the forecasted values
    forecast = test_data.copy()

    ts = train_data[["ds", "y"]].set_index("ds")
    # Fit the Simple Exponential Smoothing model
    model = SimpleExpSmoothing(ts)  # Optimize smoothing level if not provided
    model.fit(optimized=True, use_brute=True)
    yhat = model.predict(
        model.params, start=test_data["ds"].min(), end=test_data["ds"].max()
    )
    # Round the predictions to the nearest integer and ensure they are non-negative
    yhat_rounded = np.clip(np.round(yhat), a_min=0, a_max=None)

    # Assign the rounded predictions to the 'yhat' column in the forecast DataFrame
    forecast["yhat"] = yhat_rounded

    # # Calculate metrics
    mae = abs(forecast["sales"] - forecast["yhat"]).mean()
    rmse = ((forecast["sales"] - forecast["yhat"]) ** 2).mean() ** 0.5

    # Save forecast plot
    tail_df = forecast.rename(columns={"sales": "actual", "yhat": "predicted"})
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(tail_df["ds"], tail_df["actual"], color="red", label="Actual Test Data")
    ax.plot(
        tail_df["ds"],
        tail_df["predicted"],
        color="blue",
        linestyle="--",
        label="Forecasted Sales",
    )
    ax.legend()
    ax.set_title("Simple Exponential Smoothing Forecast")
    ax.set_xlabel("Date")
    ax.set_ylabel("Sales")
    fig.savefig(product_dir / "forecast_plot.png")
    plt.close(fig)

    return forecast, mae, rmse


def forecast_with_moving_average(train_data, test_data, product_dir, window=28):
    """
    Forecast sales using a moving average method dynamically over the test period
    and save the forecast plot.
    """
    # Combine train and test data for rolling mean calculation
    combined = pd.concat(
        [train_data[["ds", "y"]], test_data[["ds"]]], ignore_index=True
    )
    combined = combined.sort_values(by="ds").reset_index(drop=True)

    # Initialize the yhat list for predictions
    yhat = []

    # Dynamically calculate rolling mean for each test date
    for i in range(len(train_data), len(combined)):
        if i < window:  # Handle insufficient data for the rolling window
            rolling_window = combined.iloc[:i]["y"].mean(
                skipna=True
            )  # Use available data
        else:
            rolling_window = combined.iloc[i - window : i]["y"].mean(skipna=True)

        # Handle potential NaN by defaulting to 0
        if pd.isna(rolling_window):
            rolling_window = 0

        yhat.append(round(rolling_window))  # Ensure integer predictions

    # Assign the rolling forecast values to the test data
    forecast = test_data.copy()
    forecast["yhat"] = yhat

    # Calculate metrics
    mae = abs(forecast["sales"] - forecast["yhat"]).mean()
    rmse = ((forecast["sales"] - forecast["yhat"]) ** 2).mean() ** 0.5

    # Save forecast plot
    tail_df = forecast.rename(columns={"sales": "actual", "yhat": "predicted"})
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(tail_df["ds"], tail_df["actual"], color="red", label="Actual Test Data")
    ax.plot(
        tail_df["ds"],
        tail_df["predicted"],
        color="blue",
        linestyle="--",
        label="Forecasted Sales",
    )
    ax.legend()
    ax.set_title("28-Day Moving Average Forecast")
    ax.set_xlabel("Date")
    ax.set_ylabel("Sales")
    fig.savefig(product_dir / "forecast_plot.png")
    plt.close(fig)

    return forecast, mae, rmse


# Define a function for Prophet forecasting
def forecast_with_prophet(train_data, test_data, data_events, product_dir):
    """
    Forecast sales using Prophet and save related plots and metrics.
    """
    original_stdout = sys.stdout
    original_stderr = sys.stderr

    sys.stdout = open(os.devnull, "w")
    sys.stderr = open(os.devnull, "w")

    # Initialize and fit the Prophet model
    model = Prophet(holidays=data_events)
    model.add_country_holidays(country_name="US")
    model.add_seasonality(name="monthly", period=30.5, fourier_order=5)
    model.fit(train_data)

    sys.stdout = original_stdout
    sys.stderr = original_stderr

    # Cross-validation
    df_cv = cross_validation(model, horizon="28 days", parallel="processes")
    df_p = performance_metrics(df_cv)
    df_p.to_csv(product_dir / "performance_metrics.csv", index=False)

    # Save components and cross-validation plots
    fig = model.plot_components(model.predict(train_data))
    fig.savefig(product_dir / "components.png")
    plt.close(fig)

    fig = plot_cross_validation_metric(df_cv, metric="rmse", rolling_window=0.1)
    fig.savefig(product_dir / "cross_validation.png")
    plt.close(fig)

    # Predict test data
    forecast = model.predict(test_data)
    forecast["yhat"] = forecast["yhat"].round().clip(lower=0).astype(int)
    forecast = forecast.merge(test_data[["ds", "sales"]], on="ds", how="left")

    # Save forecast plot
    tail_df = forecast.rename(columns={"sales": "actual", "yhat": "predicted"})
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.scatter(tail_df["ds"], tail_df["actual"], color="red", label="Actual Test Data")
    ax.plot(
        tail_df["ds"],
        tail_df["predicted"],
        color="blue",
        linestyle="--",
        label="Forecasted Sales",
    )
    ax.legend()
    fig.savefig(product_dir / "forecast_plot.png")
    plt.close(fig)

    # Calculate RMSE and return
    mae = abs(forecast["sales"] - forecast["yhat"]).mean()
    rmse = ((forecast["sales"] - forecast["yhat"]) ** 2).mean() ** 0.5
    return forecast, mae, rmse


def process_product(product_id, avg_sales, threshold, days_to_analyze=365):
    # Filter data for the specified product
    train_data_filtered = merged_train_data[
        merged_train_data["id"] == product_id
    ].rename(columns={"date": "ds", "sales": "y"})
    print(f"Train Data: {train_data_filtered.shape}")

    test_data_filtered = cleaned_test_sales_data[
        cleaned_test_sales_data["id"] == product_id
    ].rename(columns={"date": "ds"})
    print(f"Test Data: {test_data_filtered.shape}")

    # Ensure the data is sorted by date
    train_data_filtered = train_data_filtered.sort_values(by="ds").reset_index(
        drop=True
    )

    # Convert 'ds' to datetime if not already
    train_data_filtered["ds"] = pd.to_datetime(train_data_filtered["ds"])

    # Filter the last 'days_to_analyze' days
    end_date = train_data_filtered["ds"].max()
    start_date = end_date - pd.Timedelta(days=days_to_analyze)
    recent_data = train_data_filtered[
        (train_data_filtered["ds"] > start_date)
        & (train_data_filtered["ds"] <= end_date)
    ]

    # Set the date as the index for decomposition
    ts = recent_data.set_index("ds")["y"]

    # Apply Box-Cox Transformation to stabilize variance
    ts_transformed, lam = boxcox(ts + 1)  # Adding 1 to avoid log(0)

    # Perform STL decomposition
    stl = STL(ts_transformed, period=7, seasonal=7)
    result = stl.fit()

    # Save the decomposition plot
    product_dir = Path(f"products/{product_id}")
    product_dir.mkdir(parents=True, exist_ok=True)

    # Perform additive decomposition
    decomposition_week = seasonal_decompose(ts, model="additive", period=7)

    # Plot the decomposed components for additive model
    fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
    axes[0].plot(ts, label="Observed", color="black")
    axes[0].set_ylabel("Observed")
    axes[0].legend(loc="upper left")
    axes[1].plot(decomposition_week.trend, label="Trend", color="blue")
    axes[1].set_ylabel("Trend")
    axes[1].legend(loc="upper left")
    axes[2].plot(decomposition_week.seasonal, label="Seasonal", color="green")
    axes[2].set_ylabel("Seasonal")
    axes[2].legend(loc="upper left")
    axes[3].plot(decomposition_week.resid, label="Residual", color="red")
    axes[3].set_ylabel("Residual")
    axes[3].legend(loc="upper left")
    axes[3].set_xlabel("Date")
    plt.tight_layout()
    plt.savefig(
        product_dir / f"last_{days_to_analyze}_train_decomposition_additive_week.png"
    )

    # Perform additive decomposition
    decomposition_monthly = seasonal_decompose(ts, model="additive", period=31)

    # Plot the decomposed components for additive model
    fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)
    axes[0].plot(ts, label="Observed", color="black")
    axes[0].set_ylabel("Observed")
    axes[0].legend(loc="upper left")
    axes[1].plot(decomposition_monthly.trend, label="Trend", color="blue")
    axes[1].set_ylabel("Trend")
    axes[1].legend(loc="upper left")
    axes[2].plot(decomposition_monthly.seasonal, label="Seasonal", color="green")
    axes[2].set_ylabel("Seasonal")
    axes[2].legend(loc="upper left")
    axes[3].plot(decomposition_monthly.resid, label="Residual", color="red")
    axes[3].set_ylabel("Residual")
    axes[3].legend(loc="upper left")
    axes[3].set_xlabel("Date")
    plt.tight_layout()
    plt.savefig(
        product_dir / f"last_{days_to_analyze}_train_decomposition_additive_monthly.png"
    )

    # Convert trend and residual components to pandas Series to handle missing values
    trend_series = pd.Series(result.trend, index=ts.index).dropna()
    resid_series = pd.Series(result.resid, index=ts.index).dropna()

    # Check for trend using Augmented Dickey-Fuller test
    adf_result = adfuller(trend_series)
    has_trend = (
        adf_result[1] > 0.05
    )  # p-value > 0.05 suggests non-stationarity (presence of trend)

    # Check for seasonality by examining the seasonal component's variance
    seasonal_variance = np.var(result.seasonal)
    has_seasonality = (
        seasonal_variance > 0.1
    )  # Threshold can be adjusted based on domain knowledge

    # Check for autocorrelation in residuals using Ljung-Box test
    lb_test = q_stat(acf(resid_series, fft=False, nlags=10), len(resid_series))
    lb_p_value = lb_test[1][-1]
    has_autocorrelation = (
        lb_p_value < 0.05
    )  # p-value < 0.05 suggests significant autocorrelation

    # Determine model complexity
    if avg_sales >= threshold and (has_trend or has_seasonality or has_autocorrelation):
        # Use Prophet method for complex patterns
        forecast, mae, rmse = forecast_with_prophet(
            train_data_filtered, test_data_filtered, data_events, product_dir
        )
    else:
        # Use Exponential Smoothing method for simpler patterns
        forecast, mae, rmse = forecast_with_exponential_smoothing(
            train_data_filtered, test_data_filtered, product_dir
        )

    # Calculate additional metrics
    test_weekly_avg = test_data_filtered["sales"].mean()
    test_total_avg = test_data_filtered["sales"].sum()
    return product_id, forecast, mae, rmse, test_weekly_avg, test_total_avg


def calculate_metrics(forecast):
    """
    Calculate evaluation metrics for the forecast.
    """
    mae = abs(forecast["sales"] - forecast["yhat"]).mean()
    rmse = ((forecast["sales"] - forecast["yhat"]) ** 2).mean() ** 0.5
    weekly_avg = forecast["sales"].mean()
    total_avg = forecast["sales"].sum()
    return mae, rmse, weekly_avg, total_avg


# Extract holidays from the calendar data
data_events = extract_holidays(calendar)

# Calculate thresholds
avg_weekly_sales = merged_train_data.groupby("id")["sales"].sum() / (
    len(merged_train_data["date"].unique()) / 7
)
THRESHOLD_PERCENTILE = 0.4
threshold = avg_weekly_sales.quantile(THRESHOLD_PERCENTILE)


# Updated results collection to include all metrics
results = [
    process_product(pid, avg_weekly_sales[pid], threshold)
    for pid in tqdm(products, desc="Processing Products", leave=True)
]

# Save metrics and submission data
output_dir = Path("products")
output_dir.mkdir(parents=True, exist_ok=True)

metrics_data = [
    {
        "Product-ID": res[0],
        "MAE": round(res[2], 2),  # Round to 2 decimals
        "RMSE": round(res[3], 2),  # Round to 2 decimals
        "Test_Weekly_Average_Sales": int(round(res[4])),  # Ensure integer for sales
        "Test_Total_Sales": int(round(res[5])),  # Ensure integer for sales
    }
    for res in results
]
metrics_df = pd.DataFrame(metrics_data)

# Save the metrics as a CSV file
metrics_df.to_csv(output_dir / "metrics.csv", index=False)


In [ ]:
# Define the root directory and metrics file path
products_dir = "products/"
metrics_path = os.path.join(products_dir, "metrics.csv")

# Read the metrics CSV
metrics_df = pd.read_csv(metrics_path)

# Initialize lists for SES and Prophet
ses_ids = []
prophet_ids = []

# Collect product IDs based on directory suffixes
for dir_name in os.listdir(products_dir):
    if dir_name.endswith("_ses"):
        ses_ids.append(dir_name.replace("_ses", ""))
    elif dir_name.endswith("_prophet"):
        prophet_ids.append(dir_name.replace("_prophet", ""))

# Filter metrics for SES and Prophet
ses_metrics = metrics_df[metrics_df["Product-ID"].str.contains("|".join(ses_ids))]
prophet_metrics = metrics_df[
    metrics_df["Product-ID"].str.contains("|".join(prophet_ids))
]

# Calculate averages for SES
ses_avg_rmse = ses_metrics["RMSE"].mean()
ses_avg_weekly_sales = ses_metrics["Test_Weekly_Average_Sales"].mean()
ses_total_sales = ses_metrics["Test_Total_Sales"].sum()

# Calculate averages for Prophet
prophet_avg_rmse = prophet_metrics["RMSE"].mean()
prophet_avg_weekly_sales = prophet_metrics["Test_Weekly_Average_Sales"].mean()
prophet_total_sales = prophet_metrics["Test_Total_Sales"].sum()

# Display the results
print("SES Metrics:")
print(f"  Average RMSE: {ses_avg_rmse:.2f}")
print(f"  Average Weekly Sales: {ses_avg_weekly_sales:.2f}")
print(f"  Total Sales: {ses_total_sales}")

print("\nProphet Metrics:")
print(f"  Average RMSE: {prophet_avg_rmse:.2f}")
print(f"  Average Weekly Sales: {prophet_avg_weekly_sales:.2f}")
print(f"  Total Sales: {prophet_total_sales}")

In [ ]:
def update_product_ids(metrics_path, products_dir, output_path):
    """
    Updates the Product-ID column in the metrics.csv file based on whether
    the product was forecasted using Prophet or SES.

    Args:
        metrics_path (str): Path to the metrics.csv file.
        products_dir (str): Path to the products directory.
        output_path (str): Path to save the updated metrics.csv file.
    """
    # Load the metrics.csv file
    metrics_df = pd.read_csv(metrics_path)

    # Create a mapping for product ID suffixes based on the products directory
    id_suffix_mapping = {}

    # Iterate through the directories in the products folder
    for dir_name in os.listdir(products_dir):
        dir_path = os.path.join(products_dir, dir_name)

        # Check if it's a directory and map the suffix
        if os.path.isdir(dir_path):
            if os.path.exists(os.path.join(dir_path, "components.png")):
                id_suffix_mapping[dir_name] = "_prophet"
            else:
                id_suffix_mapping[dir_name] = "_ses"

    # Function to update the Product-ID
    def add_suffix(product_id):
        # Extract the base ID by removing the "_validation" suffix
        base_id = product_id.replace("_validation", "")
        # Determine the suffix from the mapping
        suffix = id_suffix_mapping.get(base_id, "")
        # Append the suffix and restore the "_validation"
        return f"{base_id}{suffix}_validation" if suffix else product_id

    # Apply the transformation to the Product-ID column
    metrics_df["Product-ID"] = metrics_df["Product-ID"].apply(add_suffix)

    # Save the updated metrics.csv file
    metrics_df.to_csv(output_path, index=False)
    print(f"Updated metrics file saved to {output_path}")


# Example usage
metrics_path = "products/metrics.csv"
products_dir = "products/"
output_path = "products/updated_metrics.csv"
update_product_ids(metrics_path, products_dir, output_path)

Updated metrics file saved to products/updated_metrics.csv


In [ ]:
# Extract unique product IDs from the training data
product_ids = merged_train_data["id"].unique()

# Calculate total sales for each product and sort by descending total sales
total_sales = (
    merged_train_data.groupby("id")["sales"]
    .sum()
    .reset_index()
    .rename(columns={"sales": "total_sales"})
    .sort_values(by="total_sales", ascending=False)
)

# Display the top products by total sales (preview)
display(total_sales.head())

# Define the number of top products to analyze (set to 20 or "all" for all products)
top_k = 20

# Retrieve product IDs for the top K products
if isinstance(top_k, int):
    top_products = total_sales.nlargest(top_k, "total_sales")["id"].tolist()
elif top_k == "all":
    top_products = product_ids
else:
    raise ValueError("Invalid value for top_k. Must be an integer or 'all'.")

# Load the metrics CSV file
metrics_path = "products/metrics.csv"  # Update to your actual file path
metrics_df = pd.read_csv(metrics_path)

# Ensure column names are standardized and check for expected column
if "Product-ID" not in metrics_df.columns:
    raise KeyError("Expected column 'Product-ID' not found in metrics.csv")

# Filter the metrics for the selected top products
filtered_metrics = metrics_df[metrics_df["Product-ID"].isin(top_products)]

# Generate LaTeX table for the filtered metrics
latex_table = filtered_metrics.to_latex(
    index=False,  # Exclude the index column from the LaTeX table
    header=True,  # Include column headers
    float_format="%.2f",  # Format float values to 2 decimal places
    caption="Performance Metrics for Forecasting Models",  # Add a caption
    label="tab:performance_metrics",  # Add a reference label
    column_format="|l|c|c|c|c|",  # Define column alignment
    escape=True,  # Escape special LaTeX characters
    longtable=True,  # Use longtable for tables spanning multiple pages
)

# Print the LaTeX table for inclusion in reports
# print(latex_table)

# Save filtered metrics to a CSV file
output_path = "filtered_metrics.csv"  # Update to your desired output path
filtered_metrics.to_csv(output_path, index=False)
# print(f"Filtered metrics saved to {output_path}")

# # Display the filtered metrics (optional)
# display(filtered_metrics)

In [ ]:
def get_top_and_bottom(input_file: str, sort_by: str = "RMSE", top_n: int = 10):
    """
    Get the top N and bottom N rows from a metrics CSV file based on a specified column.

    Args:
        input_file (str): Path to the input CSV file.
        sort_by (str): The column name to sort by (default is "RMSE").
        top_n (int): Number of rows to retrieve for the top and bottom (default is 10).

    Returns:
        tuple: A tuple containing two DataFrames: (top N rows, bottom N rows).
    """
    try:
        # Load the metrics CSV file
        metrics_df = pd.read_csv(input_file)

        # Check if the specified column exists
        if sort_by not in metrics_df.columns:
            raise ValueError(f"Column '{sort_by}' not found in {input_file}")

        # Sort the DataFrame by the specified column
        sorted_metrics = metrics_df.sort_values(by=sort_by, ascending=False)

        # Get the top N and bottom N rows
        top_rows = sorted_metrics.tail(top_n)
        bottom_rows = sorted_metrics.head(top_n)

        return pd.concat([top_rows, bottom_rows])

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None


# Example usage
prophet = get_top_and_bottom("prophet_metrics.csv")
ses = get_top_and_bottom("ses_metrics.csv")

# Display the results
print("Prophet Metrics:")
display(prophet)

print("SES Metrics:")
display(ses)

Top 10 Prophet Metrics:


,Product-ID,MAE,RMSE,Test_Weekly_Average_Sales,Test_Total_Sales
92,FOODS_3_141_TX_3_validation,0.29,0.60,0,5
119,FOODS_3_199_TX_3_validation,0.21,0.60,0,6
82,FOODS_3_128_TX_3_validation,0.25,0.57,0,5
166,FOODS_3_274_TX_3_validation,0.25,0.57,0,9
117,FOODS_3_194_TX_3_validation,0.32,0.57,0,6
344,FOODS_3_577_TX_3_validation,0.25,0.57,0,6
27,FOODS_3_050_TX_3_validation,0.21,0.46,0,2
178,FOODS_3_294_TX_3_validation,0.21,0.46,0,2
25,FOODS_3_045_TX_3_validation,0.21,0.46,0,6
49,FOODS_3_082_TX_3_validation,0.14,0.38,0,3



Bottom 10 Prophet Metrics:


,Product-ID,MAE,RMSE,Test_Weekly_Average_Sales,Test_Total_Sales
55,FOODS_3_090_TX_3_validation,25.61,29.35,64,1799
264,FOODS_3_444_TX_3_validation,21.46,23.20,23,653
77,FOODS_3_120_TX_3_validation,20.14,22.67,15,409
154,FOODS_3_252_TX_3_validation,15.68,19.66,51,1440
346,FOODS_3_580_TX_3_validation,7.50,19.20,9,241
373,FOODS_3_635_TX_3_validation,14.04,17.86,7,188
350,FOODS_3_586_TX_3_validation,13.96,16.25,73,2056
179,FOODS_3_295_TX_3_validation,9.96,15.68,15,421
300,FOODS_3_501_TX_3_validation,10.61,15.65,14,391
143,FOODS_3_234_TX_3_validation,10.71,15.56,15,425



Top 10 SES Metrics:


,Product-ID,MAE,RMSE,Test_Weekly_Average_Sales,Test_Total_Sales
213,FOODS_3_539_TX_3_validation,0.04,0.19,0,1
126,FOODS_3_328_TX_3_validation,0.04,0.19,0,1
314,FOODS_3_796_TX_3_validation,0.04,0.19,0,1
313,FOODS_3_793_TX_3_validation,0.04,0.19,0,1
283,FOODS_3_713_TX_3_validation,0.04,0.19,0,1
298,FOODS_3_758_TX_3_validation,0.04,0.19,0,1
236,FOODS_3_597_TX_3_validation,0.04,0.19,0,1
309,FOODS_3_779_TX_3_validation,0.04,0.19,0,1
99,FOODS_3_260_TX_3_validation,0.04,0.19,0,1
222,FOODS_3_553_TX_3_validation,0.04,0.19,0,1



Bottom 10 SES Metrics:


,Product-ID,MAE,RMSE,Test_Weekly_Average_Sales,Test_Total_Sales
50,FOODS_3_147_TX_3_validation,2.64,4.47,3,74
294,FOODS_3_747_TX_3_validation,1.96,4.32,2,59
20,FOODS_3_047_TX_3_validation,1.96,2.32,2,67
247,FOODS_3_618_TX_3_validation,1.64,2.05,2,62
107,FOODS_3_278_TX_3_validation,1.64,1.96,2,50
52,FOODS_3_152_TX_3_validation,1.75,1.92,1,25
328,FOODS_3_827_TX_3_validation,1.50,1.83,2,42
122,FOODS_3_317_TX_3_validation,1.36,1.81,1,26
217,FOODS_3_545_TX_3_validation,1.39,1.78,2,51
241,FOODS_3_603_TX_3_validation,1.25,1.78,2,49


In [ ]:
def save_forecast_to_csv(results, output_dir):
    """
    Save the forecasted values for each product in the required format.
    Ensure all values are integers.
    """
    forecast_data = []

    for product_id, forecast, _, _, _, _ in results:
        # Extract the forecasted values (yhat) as a list and ensure they are integers
        forecast_values = forecast["yhat"].astype(int).tolist()
        # Create a row with the product_id and forecast values
        forecast_row = [product_id] + forecast_values
        forecast_data.append(forecast_row)

    # Define the headers
    headers = ["id"] + [f"F{i}" for i in range(1, 29)]  # Assuming a 28-day forecast

    # Create a DataFrame
    forecast_df = pd.DataFrame(forecast_data, columns=headers)

    # Save to CSV
    forecast_df.to_csv(output_dir / "forecast_submission.csv", index=False)


# After processing products, save the forecast in the required format
save_forecast_to_csv(results, output_dir)